#Capstone Project

###*Lawrence Sheed*

---


In [1]:
#Disable pip's bloody annoying warnings
%env _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1 

#disable python warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#install libraries needed
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install geopy
!pip install geocoder

#import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json
import folium 
from bs4 import BeautifulSoup  

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    

from sklearn.cluster import KMeans
from IPython.display import display_html
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder # import geocoder

print('Libraries imported.')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



env: _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1
Libraries imported.


In [2]:
# URL https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#Read table with soup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url).text
soup= BeautifulSoup (req,'lxml')
table = soup.find ('table')

#disable debug code to see what was up with the tables
#print (table)
#myTable=str(soup.table)
#display_html(myTable,raw=True)



In [9]:

table_contents=[]
#table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print (df.head())
print()

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)


  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

The dataframe has 15 boroughs and 103 neighborhoods.


##IMPORT THE CSV AND MERGE

In [10]:
#import the CSV
geo_coords = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo_coords.head()

#merge the two 
geo_coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
merged = pd.merge(df,geo_coords,on='PostalCode')
merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
